# 라이브러리

In [55]:
import numpy as np
import torch
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

from google.colab import drive
drive.mount('/content/drive')

device = 'cuda' if cuda.is_available() else 'cpu'
path = "/content/drive/MyDrive/21_winter/CIFAR-10/cifar-10-batches-py"
batch_size = 32
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터 불러오기

In [56]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
data_list = ["/data_batch_" + str(i) for i in range(2,6)]
train_data = unpickle(path + "/data_batch_1")
test_data = unpickle(path + "/test_batch")

train_data_img = train_data[b'data']
train_data_labels = train_data[b'labels']
for i in data_list:
  train_data = unpickle(path + i)
  train_data_img = np.concatenate((train_data_img,train_data[b'data']))
  train_data_labels += train_data[b'labels']


# 데이터 준비

In [57]:
class GH_Dataset(data.Dataset): 
    def __init__(self,X,Y):
        self.x_data = torch.from_numpy(X).type(dtype=torch.float32).resize_((10000,3,32,32))
        self.y_data = torch.tensor(Y).resize_(10000,1)

    def __len__(self): 
        return len(self.x_data)
    
    def __getitem__(self, idx): 
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

train_dataset = GH_Dataset(train_data_img,train_data_labels)
train_loader = data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset = GH_Dataset(test_data[b'data'],test_data[b'labels'])
test_loader = data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

# 모델

In [58]:
class GH(nn.Module):
    def __init__(self):
        super(GH,self).__init__()
        self.l1 = nn.Conv2d(3 ,16,3, padding = 1)
        self.l2 = nn.Conv2d(16,32,3, padding = 1) 
        self.l3 = nn.Conv2d(32,64,3, padding = 1)
        
        
        self.l4 = nn.Linear(1024,256)   
        self.l5 = nn.Linear(256,128)
        self.l6 = nn.Linear(128,10)
        self.l6 = nn.Linear(128,10)
    
    def forward(self, x):      #(n,3,32,32)
        x = F.relu(self.l1(x)) #(n,16,32,32)
        x = F.max_pool2d(x,2)  #(n,16,16,16)

        x = F.relu(self.l2(x)) #(n,32,16,16)
        x = F.max_pool2d(x,2)  #(n,32, 8, 8)

        x = F.relu(self.l3(x)) #(n,64, 8, 8)
        x = F.max_pool2d(x,2)  #(n,64, 4, 4)

        x = x.view(-1,1024)    #(n,512)
        x = F.relu(self.l4(x)) #(n,256)
        x = F.relu(self.l5(x)) #(n,128)
        return self.l6(x)      #(n,10)
    
model = GH()
model.to(device)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, torch.squeeze(target))
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch : {} | Batch Status : {}/{} ({:.0f}%) | Loss : {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, torch.squeeze(target)).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    print(f'==================\nTest set: Average loss : {test_loss:.4f}, Accuracy : {correct}/{len(test_loader.dataset)}'
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

# 모델 학습

In [59]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Tesing time: {m:.0f}m {s:.0f}s')
        
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Total time : {m:.0f}m {s: .0f}s \nModel was trained on {device}!')

Train Epoch : 1 | Batch Status : 0/10000 (0%) | Loss : 2.592330
Training time: 0m 8s
Test set: Average loss : 0.0804, Accuracy : 1015/10000(10%)
Tesing time: 0m 13s
Train Epoch : 2 | Batch Status : 0/10000 (0%) | Loss : 2.735855
Training time: 0m 8s
Test set: Average loss : 0.0804, Accuracy : 1015/10000(10%)
Tesing time: 0m 12s
Train Epoch : 3 | Batch Status : 0/10000 (0%) | Loss : 2.699316
Training time: 0m 8s
Test set: Average loss : 0.0804, Accuracy : 1015/10000(10%)
Tesing time: 0m 12s
Train Epoch : 4 | Batch Status : 0/10000 (0%) | Loss : 2.717904
Training time: 0m 8s
Test set: Average loss : 0.0804, Accuracy : 1015/10000(10%)
Tesing time: 0m 12s
Train Epoch : 5 | Batch Status : 0/10000 (0%) | Loss : 2.741710
Training time: 0m 8s
Test set: Average loss : 0.0804, Accuracy : 1015/10000(10%)
Tesing time: 0m 12s
Train Epoch : 6 | Batch Status : 0/10000 (0%) | Loss : 2.361216
Training time: 0m 9s
Test set: Average loss : 0.0804, Accuracy : 1015/10000(10%)
Tesing time: 0m 13s
Train Epoc